In [20]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import operator


In [71]:
from text_processing import get_text_map


C:\Users\Asus\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [72]:
texts_lenta = pd.read_csv("./colloc/3000.csv")

texts_lenta.iloc[69]['texts_3000']

'Судебные приставы во вторник начали арест имущества госпредприятия "Красноярская железная дорога", исполняя решения краевого арбитражного суда по искам на сумму 944 миллионов рублей. Основными кредиторами Красноярской железной дороги являются Красноярский и Саянский алюминиевые заводы. В арбитражном суде Красноярского края подтвердили, что начат арест офисной мебели Красноярской железной дороги, оргтехники, а также части автотранспорта. По заявлению начальника юридической службы Красноярской железной дороги Вениамина Литвина, его предприятие, в свою очередь, обратилось в арбитражный суд с обжалованием действий приставов. Руководство дороги, в частности, считает, что арест автотранспорта может привести к остановке некоторых подразделений предприятия, передает "Интерфакс".'

In [73]:
examined_text = get_text_map(texts_lenta.iloc[69]['texts_3000'], raw_text_input=True)

In [60]:
text_db = pd.read_csv("PUPKIN_ZALUP_text_db.csv",header = None)
text_db.head()

,0,1,2,3,4,5,6,7,8,9
0,0.7722,0.74,0.000000,0.000000,0.035649,0.047113,0.000000,0.141178,0.251913,0.75
1,0.6010,0.61,0.500000,0.300000,0.021810,0.036228,0.003830,0.077616,0.241760,0.75
2,0.6750,0.61,0.285714,0.357143,0.024583,0.004887,0.020406,0.103192,0.221809,0.75
3,0.6261,0.73,0.000000,0.142857,0.040532,0.008322,0.007678,0.121881,0.228108,0.75
4,0.6974,0.70,0.125000,0.125000,0.031904,0.007822,0.000000,0.108833,0.213396,0.75


In [61]:
sent_db = pd.read_csv("PUPKIN_ZALUP_sentence_db.csv",header = None)
sent_db.head()

,0,1,2,3,4,5,6,7
0,0.0,0.000000,0.000000,0.121385,0.0,0.121385,0.200000,1.0
1,0.0,0.000000,0.018055,0.036111,0.0,0.000000,0.389362,0.0
2,0.0,0.000000,0.131344,0.000000,0.0,0.237312,0.177778,1.0
3,0.0,0.000000,0.097245,0.097245,0.0,0.319285,0.283333,1.0
4,0.0,0.037037,0.042854,0.085708,0.0,0.085708,0.284615,0.0


In [62]:
word_db = pd.read_csv("PUPKIN_ZALUP_word_db.csv",header = None)
word_db.head()

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,300,301,302
0,9.0,176.85,0.063438,0.061131,0.110517,0.019039,0.026680,0.075326,0.054573,-0.031124,...,0.070271,-0.039740,0.085476,0.009615,-0.026628,-0.015571,-0.032359,-0.044872,-0.103097,1.0
1,602.0,952.20,0.022752,0.106342,-0.062723,0.088502,-0.012682,0.017010,0.095227,-0.062190,...,-0.040335,0.016047,0.119008,0.022371,0.042570,-0.027927,-0.075974,-0.071021,0.012131,1.0
2,23.0,154.40,-0.004272,0.031035,-0.040596,0.019975,0.061630,0.140724,-0.012883,-0.007610,...,-0.057840,-0.017444,-0.122268,0.081686,-0.023513,-0.037689,0.010409,-0.029327,0.045051,1.0
3,6.0,59.60,-0.011012,0.054919,0.023132,-0.070816,0.045225,0.073546,0.068018,-0.057633,...,0.011395,-0.029004,-0.047611,0.050858,-0.081211,-0.036254,-0.014185,-0.058991,-0.020729,1.0
4,44.0,27.80,-0.048169,0.072897,0.007602,0.054211,-0.098255,0.087783,-0.026557,0.015555,...,-0.012440,-0.016374,0.032690,0.018083,0.055473,-0.022616,-0.072336,0.037692,0.045525,1.0


with open ("text_map_improved_example.json", "r", encoding = "utf-8") as f:
    examined_text = json.load(f)

In [80]:
words_dict = []
with open("colloc_lit.txt", "r", encoding = "utf-8") as f:
    for line in f.readlines():
        words_dict.append(line[:-1])

In [81]:
words_dict[0]

'указ президент'

In [46]:
my_list = [ ('John', 40), ('Dana', 35), ('Betty', 10), ('Robby', 8), ('John', 20) ]
my_list.sort(key = operator.itemgetter(1))
print(my_list)

[('Robby', 8), ('Betty', 10), ('John', 20), ('Dana', 35), ('John', 40)]


In [99]:
def get_top_similar_vectors_from_marked_db(analyzed_text_map,marked_text_db, marked_sent_db, marked_word_db, words_indexed_dict):
    
    similarity_json = {"similar_texts":[],"simiar_sentences":[], "similar_collocations":[]}
    #text_similarity
    text_vector = []
    text_vector.append(analyzed_text_map['lix'])
    text_vector.append(analyzed_text_map['ttr'])
    text_vector.extend(analyzed_text_map['sent_properties'])
    list_of_texts_similarities = []
    for text_ind in range(len(marked_text_db)):
        #print(text_vector)
        #target_var = list(marked_text_db.iloc[2][-1:]) WILL NEED WHEN PREDICT!
        v1 = np.array(text_vector).reshape(1, -1)
        v2 = np.array(list(marked_text_db.iloc[text_ind][:-1])).reshape(1, -1)
        sim = cosine_similarity(v1,v2)
        #print(sim[0][0])
        if sim[0][0] > 0.6:
            list_of_texts_similarities.append((sim[0][0], {"TEXT_IND_in_marked_db":text_ind, "VECTOR":list(marked_text_db.iloc[text_ind][:-1])}))
            
    list_of_texts_similarities.sort(key = operator.itemgetter(0), reverse= True)   
    similarity_json['similar_texts'] = list_of_texts_similarities[:5]
    
    list_of_all_sentences_similarity = []
    for sentence in analyzed_text_map['sentences_map']:
        current_sentence = []
        current_sentence.append(sentence['spec_sentence_features']['negation'])
        current_sentence.append(sentence['spec_sentence_features']['coreference'])
        current_sentence.append(sentence['spec_sentence_features']['vozvr_verb'])
        current_sentence.append(sentence['spec_sentence_features']['prich'])
        current_sentence.append(sentence['spec_sentence_features']['deepr'])
        current_sentence.append(sentence['spec_sentence_features']['case_complexity'])
        current_sentence.append(sentence['spec_sentence_features']['mean_depend_length'])
        v1 = np.array(current_sentence).reshape(1, -1)
        list_of_current_sent_similarities = []
        for sent_ind in range(len(marked_sent_db)):
            v2 = np.array(list(marked_sent_db.iloc[sent_ind][:-1])).reshape(1, -1)
            sim = cosine_similarity(v1,v2)
            if sim[0][0] > 0.6:
                list_of_current_sent_similarities.append((sim[0][0], {"sent_ind_in_marked_db":sent_ind, "sent_vector":list(marked_sent_db.iloc[sent_ind][:-1])}))
        list_of_current_sent_similarities.sort(key = operator.itemgetter(0), reverse= True)
        list_of_all_sentences_similarity.append(list_of_current_sent_similarities[:5])
    similarity_json['simiar_sentences'] = list_of_all_sentences_similarity
    
    list_of_all_words_similarity = []
    for sentence in analyzed_text_map['sentences_map']:
        for colloc in sentence['collocation_vectors_list']:
            current_word_similarity = []
            #print(colloc[1][0])
            freq_vec = np.array([colloc[1][1], colloc[1][2]]).reshape(1,-1)
            colloc_w2v = np.array(colloc[2]).reshape(1,-1)
            for db_word_index in range(len(marked_word_db)):
                compared_raw_word_vect = marked_word_db.iloc[db_word_index][:-1]
                
                comp_word2vec = compared_raw_word_vect[2:]
                #print(db_word_index, len(comp_word2vec))
                comp_colloc_w2v = np.array(comp_word2vec).reshape(1,-1)
                w2v_sim = cosine_similarity(colloc_w2v,comp_colloc_w2v)
                #print("w2v", sim)
                
               
                comp_freq_vec = np.array([compared_raw_word_vect[0], compared_raw_word_vect[1]]).reshape(1,-1)
                sim_freq = cosine_similarity(freq_vec,comp_freq_vec)
                #print("freq", sim_freq)
                
                average_similarity = w2v_sim*0.8 + sim_freq*0.2
                
                word_comp_json = {"colloc_db_index":db_word_index, "sim_colloc":words_indexed_dict[db_word_index],"w2v_sim":w2v_sim,"freqence_similarity":sim_freq}
                if average_similarity[0][0] > 0.6:
                    current_word_similarity.append((word_comp_json,average_similarity[0][0]))
            current_word_similarity.sort(key = operator.itemgetter(1), reverse= True)
            list_of_all_words_similarity.append((colloc[1][0],current_word_similarity[:5]))
    similarity_json['similar_collocations'] = list_of_all_words_similarity
    return similarity_json
    
text_sim_list = get_top_similar_vectors_from_marked_db(examined_text,text_db,sent_db,word_db,words_dict)

In [100]:
text_sim_list['similar_texts']

[(0.998322153409185,
  {'TEXT_IND_in_marked_db': 0,
   'VECTOR': [0.7722,
    0.74,
    0.0,
    0.0,
    0.035648662266713274,
    0.047112919747615674,
    0.0,
    0.14117766568625106,
    0.2519127444659359]}),
 (0.9830769119997174,
  {'TEXT_IND_in_marked_db': 3,
   'VECTOR': [0.6261,
    0.73,
    0.0,
    0.14285714285714288,
    0.04053150174607168,
    0.008322019732960737,
    0.007677541236674239,
    0.12188090865846045,
    0.22810841332580464]}),
 (0.9827810275509946,
  {'TEXT_IND_in_marked_db': 4,
   'VECTOR': [0.6973999999999999,
    0.7,
    0.125,
    0.125,
    0.0319036509146643,
    0.007821584309844734,
    0.0,
    0.10883307585026733,
    0.21339601510460643]}),
 (0.9093564554953111,
  {'TEXT_IND_in_marked_db': 6,
   'VECTOR': [0.66,
    0.88,
    0.33333333333333326,
    0.33333333333333326,
    0.0,
    0.02423591004792057,
    0.0,
    0.26625251320409576,
    0.19345238095238093]}),
 (0.9074627854285235,
  {'TEXT_IND_in_marked_db': 8,
   'VECTOR': [0.6646,
  

In [101]:
text_sim_list['simiar_sentences']

[[(0.9943141470306037,
   {'sent_ind_in_marked_db': 11,
    'sent_vector': [0.0,
     0.0,
     0.0,
     0.0,
     0.04820596009983599,
     0.215295156612925,
     0.2681818181818182]}),
  (0.9755870826610982,
   {'sent_ind_in_marked_db': 23,
    'sent_vector': [0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.16763717627816047,
     0.20909090909090913]}),
  (0.961723028095169,
   {'sent_ind_in_marked_db': 24,
    'sent_vector': [0.0, 0.0, 0.0, 0.0, 0.0, 0.2633822389785826, 0.1875]}),
  (0.9514024669248512,
   {'sent_ind_in_marked_db': 20,
    'sent_vector': [0.04166666666666666,
     0.04166666666666666,
     0.0,
     0.0,
     0.0,
     0.16142814751883036,
     0.19130434782608696]}),
  (0.9512451120129124,
   {'sent_ind_in_marked_db': 19,
    'sent_vector': [0.0,
     0.0,
     0.07711125730591169,
     0.0,
     0.0,
     0.2194381156914192,
     0.2266666666666667]})],
 [(0.9835568457007837,
   {'sent_ind_in_marked_db': 18,
    'sent_vector': [0.0,
     0.0,
     0.0570714

In [102]:
for sim in text_sim_list['similar_collocations']:
    print (sim)
    print()

('судебный пристава', [({'colloc_db_index': 42, 'sim_colloc': 'решение суд правоохранительный', 'w2v_sim': array([[0.72743916]]), 'freqence_similarity': array([[0.99992414]])}, 0.7819361557650264), ({'colloc_db_index': 45, 'sim_colloc': 'суд', 'w2v_sim': array([[0.66688301]]), 'freqence_similarity': array([[0.91653038]])}, 0.7168124792965904)])

('в', [({'colloc_db_index': 22, 'sim_colloc': 'в', 'w2v_sim': array([[1.]]), 'freqence_similarity': array([[1.]])}, 1.0000000000000004), ({'colloc_db_index': 24, 'sim_colloc': 'в', 'w2v_sim': array([[1.]]), 'freqence_similarity': array([[1.]])}, 1.0000000000000004), ({'colloc_db_index': 73, 'sim_colloc': 'в', 'w2v_sim': array([[1.]]), 'freqence_similarity': array([[1.]])}, 1.0000000000000004), ({'colloc_db_index': 141, 'sim_colloc': 'в', 'w2v_sim': array([[1.]]), 'freqence_similarity': array([[1.]])}, 1.0000000000000004), ({'colloc_db_index': 146, 'sim_colloc': 'в', 'w2v_sim': array([[1.]]), 'freqence_similarity': array([[1.]])}, 1.000000000000

# В ПРОЦЕСЕ РЕКОММЕНДАЦИИ ПОЛЬЗОВАТЬСЯ ИНДЕКСАМИ РАЗМЕЧЕННЫХ ПРЕДЛОЖЕНИЙ ИЗ БАЗЫ ДАННЫХ
ТО ЕСТЬ В КАРТАХ ИМИЛАРИТИ ДОЛЖНЫ БЫТЬ СРАЗУ ИНДЕКСЫ ВСЕЙ ХЕРНИ

In [45]:
for text in text_sim_list:
    if text[1] == text_vector:
        print("yes")

yes
